<a href="https://colab.research.google.com/github/cmlins/recuperacaoDeInformacao/blob/master/proc_consulta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import math

from bs4 import BeautifulSoup as bs
from bs4 import UnicodeDammit

In [167]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Funções

## Parse HTML

In [0]:
symbolsToReplace = [
    '(', ')', '!', '?', ':', ';', '*', '.', ',', '★', '|', '+', '[', ']', '{', '}', '/', 'ª', 'º', '°', '-',
    '%', '—', '@', '#',  '\"','\'', '<', '>', '=', '´', '`', '“', '$', '&', '’', '¡', '€', 'µ', '¦',
    '\\', '®', '™', '”', '…', '‘', '•', '😍','😙', '❤', '–', '⭐️', '_', '️⭐', '😉', '👏', '¹', '²', '³',
    "£", '¢', '¬', '§', '~', '^', '×', '÷',
    '\u200c', '\u200f', '\u200e', '\t', '\xa0', '\x03', '\u0301', '\u0303'
]

allowedSmallTokens = [
  'fio', 'box', 'god', 'key', 'bit', 'led', 'faq', 'dog', 'sim', 'ovo', 'pão', 'aba', 'new'
]

In [0]:
def getAndCleanHTML(filepath):
  
  page = open(filepath, "rb")

  page = page.read()

  clearPage = UnicodeDammit.detwingle(page)
  
  doc = bs(clearPage, 'lxml')
  
  for script in doc(["script", "style"]):
    script.extract()
  
  docText = doc.get_text(' ')
  
  docText = docText.translate({ord(ch): None for ch in '0123456789'})
  
  docText = docText.lower()
  
  docText = docText.replace("e-book", "ebook")

  docText = docText.replace("blu-ray", "bluray")
  
  for symbol in symbolsToReplace:
    docText = docText.replace(symbol, ' ')
  
  docText = docText.replace("  ", ' ')
  
  lines = (line.strip() for line in docText.splitlines())
  
  chunks = (phrase.strip() for line in lines for phrase in line.split(' '))

  text = ' '.join(chunk for chunk in chunks if chunk)
  
  return text

## Searching Index

In [0]:
def handlingQuery(query):
  hq = query.split()
  return hq

In [0]:
def search(q, indice):
  hq = handlingQuery(q)
  df_search = pd.DataFrame()
  for q in hq:
    for w in df_indice['Terms'].values:
      if w in q:
        df_search = df_search.append(df_indice[df_indice['Terms'] == w], ignore_index=True)
  return df_search

In [0]:
def docs_list(df):
  postings = list()
  for r in df.index:
    postings.append(df.loc[r,'Posting'])

  for i in range(len(postings)):
    new_postings = postings[i].split(', ')
    new_postings[0] = new_postings[0][1:]
    new_postings[len(new_postings)-1] = new_postings[len(new_postings)-1][:-1]
    postings[i] = new_postings

  p = list()
  for i in range(len(postings)):
    for j in range(len(postings[i])):
      p.append(postings[i][j])

  p = list(dict.fromkeys(p))
  return p

## Calc Scores

In [0]:
def calc_tfidf(df_freq):
  for r in df_freq.index:
    df_freq.at[r, 'Freq'] = math.log(df_arquivos.shape[0]/df_freq.at[r, 'Freq'],10)
    for c in range(2, len(df_freq.columns)):
      df_freq.iloc[r, c] = np.multiply(df_freq.at[r, 'Freq'], df_freq.iloc[r, c])

  return df_freq

In [0]:
def calc_freq(dl):
  df_freq = pd.DataFrame(columns=dl)
  df_freq = pd.concat([df_s['Terms'], df_s['Freq'], df_freq], axis=1)

  for r in df_freq.index:
    for c in range(2, len(df_freq.columns)):
      df_freq.iloc[r, c] = 0

  df_freq.iloc[:, 1:] = df_freq.iloc[:, 1:].astype(float)

  queryl = handlingQuery(q)
  for i in range(len(dl)):
    doc = getAndCleanHTML(df_arquivos.loc[int(dl[i]),'0'])
    doc = doc.split()
    for w in doc:
      for qw in range(len(queryl)):
        if w in queryl[qw]:
          df_freq.at[qw, dl[i]] += 1.0

  df_freq.iloc[:, 2:] = np.log10(df_freq.iloc[:, 2:])
  df_freq = calc_tfidf(df_freq)
  return df_freq

## Calc Ranking

In [0]:
def calc_rank(df_freq, hq):
  df_freq.at[len(df_freq.index),'Terms'] = 'cosine'
  df_freq.index = df_freq['Terms']
  df_freq = df_freq.iloc[:, 1:]

  dv = []
  qv = []
  for q in hq:
    qv.append(1)

  for c in range(2, len(df_freq.columns)):
    for r in range(len(df_freq.index[:-1])):
      dv.append(df_freq.iloc[r,c])
    df_freq.iloc[len(df_freq.index)-1, c] = (np.dot(dv, qv) / (np.linalg.norm(dv) * np.linalg.norm(qv)))
    dv = []

  df_rank = df_freq.sort_values(by='cosine', axis=1, ascending=False)
  ranking = df_rank.columns.tolist()
  ranking = ranking[:10]
  return ranking

# Main

In [0]:
df_indice = pd.read_csv('/content/drive/My Drive/indice_invertido.csv')

In [0]:
df_arquivos = pd.read_csv('/content/drive/My Drive/positiveFilesList.csv')

In [182]:
q = 'vidas secas'
df_s = search(q,df_indice)
df_s.head()

,Terms,Freq,Posting,Compress Posting
0,vida,103,"[0, 1, 3, 13, 14, 15, 16, 17, 20, 21, 25, 29, ...","[0, 1, 2, 10, 1, 1, 1, 1, 3, 1, 4, 4, 1, 1, 1,..."
1,seca,26,"[85, 88, 104, 125, 146, 147, 148, 149, 150, 15...","[85, 3, 16, 21, 21, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."


In [183]:
dl = docs_list(df_s)
print(dl)

['0', '1', '3', '13', '14', '15', '16', '17', '20', '21', '25', '29', '30', '31', '32', '33', '35', '36', '37', '44', '47', '56', '57', '59', '60', '61', '62', '64', '65', '66', '68', '69', '72', '74', '78', '79', '80', '81', '84', '85', '87', '88', '89', '90', '91', '92', '93', '95', '96', '98', '99', '101', '102', '104', '106', '107', '112', '113', '114', '119', '120', '121', '122', '123', '125', '126', '130', '131', '134', '136', '137', '138', '140', '143', '145', '146', '147', '149', '150', '153', '154', '155', '157', '160', '162', '163', '165', '168', '172', '177', '185', '187', '190', '197', '199', '207', '208', '216', '225', '227', '233', '234', '235', '148', '151', '152', '156', '158', '159', '161', '164', '166', '167']


In [184]:
df_f = calc_freq(dl)
df_f.head()

,Terms,Freq,0,1,3,13,14,15,16,17,20,21,25,29,30,31,32,33,35,36,37,44,47,56,57,59,60,61,62,64,65,66,68,69,72,74,78,79,80,81,...,143,145,146,147,149,150,153,154,155,157,160,162,163,165,168,172,177,185,187,190,197,199,207,208,216,225,227,233,234,235,148,151,152,156,158,159,161,164,166,167
0,vida,0.374553,0.700126,0.558593,0.688746,0.708689,0.728311,0.683961,0.660497,0.755485,0.764609,0.768986,0.771840,0.763123,0.673948,0.737300,0.740762,0.755485,0.639575,0.691086,0.607992,0.912471,0.495241,0.536121,0.429285,0.529982,0.510039,0.542037,0.510039,0.591712,0.542037,0.611820,0.573619,0.578335,0.516962,0.782779,0.766081,0.787985,0.832524,0.797922,...,0.568763,0.558593,0.636354,0.648873,0.642734,0.578335,0.611820,0.642734,0.645832,0.708689,0.657667,0.607992,0.679031,0.651857,0.404210,0.529982,0.429285,0.440508,0.470166,0.429285,0.417230,0.478960,0.495241,0.573619,0.516962,0.523602,0.529982,0.502808,0.523602,0.516962,0.578335,0.683961,0.626289,0.529982,0.639575,0.688746,0.693394,0.639575,0.619215,0.622790
1,seca,0.972416,2.028811,1.800194,2.039070,2.296583,2.336834,2.296583,2.331796,2.405969,2.266133,2.163927,2.266133,2.226867,2.077782,2.145954,2.183574,2.110398,2.035678,2.071572,1.895619,2.617711,1.463633,1.422060,1.722006,1.817672,1.800194,1.845284,1.781962,1.905004,1.876199,1.834455,1.676714,1.722006,1.578474,2.171402,2.237559,2.237559,2.241762,2.289174,...,1.775705,1.781962,1.977335,1.957316,2.000168,1.940589,1.949035,2.071572,2.035678,2.101750,2.035678,1.981227,2.055633,2.049086,0.821787,1.083217,1.049414,1.143651,1.083217,1.220648,1.170906,1.699967,1.692357,1.855842,1.699967,1.501477,1.588412,1.407240,1.588412,1.489235,1.931970,2.101750,1.988906,1.861024,2.042435,2.104652,2.028811,2.011138,1.985084,1.957316


In [185]:
r = calc_rank(df_f, hq)
print(r)

['168', '172', '187', '177', '122', '185', '56', '120', '131', '80']
